In [2]:
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.decomposition import NMF, TruncatedSVD
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.pipeline import FeatureUnion
import pandas as pd
import numpy as np

In [60]:
from sklearn.naive_bayes import MultinomialNB

1. Загрузите датасет. Описание датасета можно посмотреть здесь.

Считайте датасет в Python (можете сразу грузить все в память, выборка небольшая), выясните, что используется в качестве разделителей и как проставляются метки классов.

In [9]:
with open ('SMSSpamCollection.txt','r', encoding='CP1252') as f:
    sms = f.read().splitlines()

2. Подготовьте для дальнейшей работы два списка: список текстов в порядке их следования в датасете и список соответствующих им меток классов. В качестве метки класса используйте 1 для спама и 0 для "не спама".

In [48]:
sms_label, sms_text = zip(*(line.split('\t') for line in sms))
sms_label_bi = np.array([1 if label== 'spam' else 0 for label in sms_label]) 

In [40]:
print (sms_label_bi[:10])
print (sms_text[:10])

[0 0 1 0 0 1 0 0 1 1]
('Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...', 'Ok lar... Joking wif u oni...', "Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's", 'U dun say so early hor... U c already then say...', "Nah I don't think he goes to usf, he lives around here though", "FreeMsg Hey there darling it's been 3 week's now and no word back! I'd like some fun you up for it still? Tb ok! XxX std chgs to send, Â£1.50 to rcv", 'Even my brother is not like to speak with me. They treat me like aids patent.', "As per your request 'Melle Melle (Oru Minnaminunginte Nurungu Vettam)' has been set as your callertune for all Callers. Press *9 to copy your friends Callertune", 'WINNER!! As a valued network customer you have been selected to receivea Â£900 prize reward! To claim call 09061701461. Claim code KL341. Valid 12 hours o

3. Используя sklearn.feature_extraction.text.CountVectorizer со стандартными настройками, получите из списка текстов матрицу признаков X.

In [41]:
count = CountVectorizer()
X = count.fit_transform(sms_text)
X.shape

(5574, 8752)

4. Оцените качество классификации текстов с помощью LogisticRegression() с параметрами по умолчанию, используя sklearn.cross_validation.cross_val_score и посчитав среднее арифметическое качества на отдельных fold'ах. Установите random_state=2. Параметр cv задайте равным 10. В качестве метрики качества используйте f1-меру. Получившееся качество - один из ответов, которые потребуются при сдаче задания. Ответ округлить до 1 знака после запятой.X

In [42]:
pipeline = Pipeline(
    [('count', CountVectorizer()),
    ('log_res', LogisticRegression())
      ])

In [45]:
score = cross_val_score(pipeline, sms_text, sms_label_bi, scoring = 'f1', cv=10).mean()
score

0.9318858273157098

In [44]:
with open('answer1.txt','w') as f:
    f.write(str(round(score, 1)))

 5. А теперь обучите классификатор на всей выборке и спрогнозируйте с его помощью класс для следующих сообщений:

"FreeMsg: Txt: CALL to No: 86888 & claim your reward of 3 hours talk time to use from your phone now! Subscribe6GB"

"FreeMsg: Txt: claim your reward of 3 hours talk time"

"Have you visited the last lecture on physics?"

"Have you visited the last lecture on physics? Just buy this book and you will have all materials! Only 99$"

"Only 99$"

Прогнозы классификатора (0 - не спам, 1 - спам), записанные через пробел, будут ответом в одном из вопросов ниже.

In [49]:
sms_test = ['FreeMsg: Txt: CALL to No: 86888 & claim your reward of 3 hours talk time to use from your phone now! Subscribe6GB',
           'FreeMsg: Txt: claim your reward of 3 hours talk time',
           'Have you visited the last lecture on physics?',
           'Have you visited the last lecture on physics? Just buy this book and you will have all materials! Only 99$',
           'Only 99$']

In [50]:
pipeline.fit(sms_text, sms_label_bi)

Pipeline(memory=None,
         steps=[('count',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=None)),
                ('log_res',
                 LogisticRegression(C=1.0, class_weight=None, dual=False,
                                    fit_intercept=True, intercept_scaling=1,
                                    l1_ratio=None, max_iter=100,
                                    multi_class='auto', n_jobs=None,
                     

In [51]:
test_predict = pipeline.predict(sms_test)
test_predict

array([1, 1, 0, 0, 0])

In [53]:
with open ('answer2.txt','w') as f:
    f.write(' '.join(map(str, test_predict)))

6. Задайте в CountVectorizer параметр ngram_range=(2,2), затем ngram_range=(3,3), затем ngram_range=(1,3). Во всех трех случаях измерьте получившееся в кросс-валидации значение f1-меры, округлите до второго знака после точки, и выпишете результаты через пробел в том же порядке. В данном эксперименте мы пробовали добавлять в признаки n-граммы для разных диапазонов n - только биграммы, только триграммы, и, наконец, все вместе - униграммы, биграммы и триграммы. Обратите внимание, что статистики по биграммам и триграммам намного меньше, поэтому классификатор только на них работает хуже. В то же время это не ухудшает результат сколько-нибудь существенно, если добавлять их вместе с униграммами, т.к. за счет регуляризации линейный классификатор не склонен сильно переобучаться на этих признаках.

In [66]:
ngram_range = [(2,2), (3,3), (1,3)]
scores = []
for ngram in ngram_range:
    pipeline = Pipeline([('count', CountVectorizer(ngram_range = ngram)), ('log_res', LogisticRegression())])
    score = cross_val_score(pipeline, sms_text, sms_label_bi, scoring = 'f1', cv=10).mean()
    scores.append(round(score, 2))
    print ('ngram_range:', ngram)
    print ('score:', round(score, 2))

ngram_range: (2, 2)
score: 0.82
ngram_range: (3, 3)
score: 0.72
ngram_range: (1, 3)
score: 0.92


In [67]:
with open ('answer3.txt','w') as f:
    f.write(' '.join(map(str, scores)))

7. Повторите аналогичный п.7 эксперимент, используя вместо логистической регрессии MultinomialNB(). Обратите внимание, насколько сильнее (по сравнению с линейным классификатором) наивный Байес страдает от нехватки статистики по биграммам и триграммам.

По какой-то причине обучение наивного байесовского классификатора через Pipeline происходит с ошибкой. Чтобы получить правильный ответ, отдельно посчитайте частоты слов и обучите классификатор.

In [74]:
scores2 = []
for ngram in ngram_range:
    #pipeline = Pipeline([('count', CountVectorizer(ngram_range = ngram)), ('NB', MultinomialNB())])
    cls = CountVectorizer(ngram_range = ngram)
    X = cls.fit_transform(sms_text)
    score2 = cross_val_score(MultinomialNB(), X, sms_label_bi, scoring = 'f1', cv=10).mean()
    scores2.append(round(score2, 2))
    print ('ngram_range:', ngram)
    print ('score:', round(score2, 2))

ngram_range: (2, 2)
score: 0.64
ngram_range: (3, 3)
score: 0.38
ngram_range: (1, 3)
score: 0.89


In [75]:
with open ('answer4.txt','w') as f:
    f.write(' '.join(map(str, scores2)))

8. Попробуйте использовать в логистической регрессии в качестве признаков Tf*idf из TfidfVectorizer на униграммах. Повысилось или понизилось качество на кросс-валидации по сравнению с CountVectorizer на униграммах? (напишите в файле с ответом 1, если повысилось, -1, если понизилось, и 0, если изменилось не более чем на 0.01). Обратите внимание, что результат перехода к tf*idf не всегда будет таким - если вы наблюдаете какое-то явление на одном датасете, не надо сразу же его обобщать на любые данные

In [70]:
pipeline = Pipeline([('tfidf', TfidfVectorizer()), ('NB', MultinomialNB())])

In [71]:
score_tf = cross_val_score(pipeline, sms_text, sms_label_bi, scoring = 'f1', cv=10).mean()
score_tf

0.8301827808377353

In [72]:
with open ('answer5.txt','w') as f:
    f.write('-1')